![](../assets/placeholder.png)

## Introduction
This notebook is designed to guide you through the process of building tools for agents using Langchain. Tools are a key part of agents and a great way to add more capabilities to an agent and the agent can combine these in useful ways. In this notebook, we will explore how to create custom tools and integrate them with agents to enhance their capabilities.

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [13]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=10000)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [14]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [15]:
tool.args

{'query': {'title': 'Query',
  'description': 'query to look up on wikipedia',
  'type': 'string'}}

In [16]:
tool.run("AI agents")

'Page: Intelligent agent\nSummary: In intelligence and artificial intelligence, an intelligent agent (IA) is an agent acting in an intelligent manner. It perceives its environment, takes actions autonomously in order to achieve goals, and may improve its performance with learning or acquiring knowledge. An intelligent agent may be simple or complex: A thermostat or other control system is considered an example of an intelligent agent, as is a human being, as is any system that meets the definition, such as a firm, a state, or a biome.\n\nLeading AI textbooks define "artificial intelligence" as the "study and design of intelligent agents", a definition that considers goal-directed behavior to be the essence of intelligence. Goal-directed agents are also described using a term borrowed from economics, "rational agent".\nAn agent has an "objective function" that encapsulates all the IA\'s goals. Such an agent is designed to create and execute whatever plan will, upon completion, maximize 

#### Custom tools
We don't just have to use the default tools from LangChain. We can define our own custom tools. A tool can be any method that has clearly defined inputs and outputs so that the agent can invoke it. Function that adds two numbers together, it can be a tool. Function that calls a custom company API, it can be a tool. Function that generates code from a custom LLM, it can be a tool. This really opens up a lot of opportunities to extend an agent with many capabilities. In this section, we will explore how to define and integrate custom tools with agents.

In [20]:
from langchain.tools import tool

# tool decorator is the easiest way to define a new tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [21]:
tool = multiply

In [29]:
tool.run({'a':65, 'b':67})

4355

### Web retriever tool
Let's build a tool that can fetch the content of a URL, this is something we've had to do in multiple previous notebooks. This tool will be particularly useful for agents that need to gather information from the web to answer questions or perform tasks.

In [37]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.tools import tool

@tool
def fetch_url_content(url: str) -> str:
    """Use this tool to fetch content of a URL"""
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs

In [54]:
tool = fetch_url_content
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
content = tool.run({'url': url})
content = str(content)

#### Summarizer tool
Let's convert what we did in summarize notebook into a tool too. This tool will enable agents to generate concise summaries of text content, which can be useful for quickly understanding the gist of large documents or web pages.

In [55]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.tools import tool
from langchain_openai import ChatOpenAI

@tool
def summarize_content(content: str) -> str:
    """Use this tool to generate the summary of a text based content"""
    system_message = "Please provide a concise summary for the content"
    human_message = "Content: " + str(content)
    
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=human_message),
    ]
    model = ChatOpenAI(model="gpt-4o")
    parser = StrOutputParser()
    
    chain = model | parser
    summary = chain.invoke(messages)
    return summary

In [56]:
tool = summarize_content
tool.run({"content": content})

'The document titled "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, discusses the construction of autonomous agents using large language models (LLMs) as core controllers. It provides an in-depth overview of the system\'s key components: planning, memory, and tool use. The planning component involves task decomposition and self-reflection. Memory is divided into short-term and long-term, facilitated by techniques like Maximum Inner Product Search (MIPS). Tool use extends LLM capabilities by integrating external APIs. The article also covers case studies such as ChemCrow and Generative Agents and mentions challenges in context length, planning, and reliability. Various proof-of-concept examples like AutoGPT and GPT-Engineer are highlighted, demonstrating the potential and limitations of LLM-powered autonomous agents.'

We've managed to take the output from one tool and pass it to another. Which is actually super simple with just regular programming with functions. Wouldn't it be great if the AI can just figure out when this needs to happen and make decisions about different permutations across many tools to complete a task? That is what AI agents exactly do. They are able to run tools or generations in a loop on their own until the task is completed. That is what we're going to dive into next!